In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [10]:
df = pd.read_csv("AirQualityUCI.csv", sep=';', na_values=-200, decimal=',')

# 1. Базовая линейная регрессия с одним признаком

In [ ]:
filtered_df = df[df['CO(GT)'].notnull()]
filtered_df = filtered_df[filtered_df['C6H6(GT)'].notnull()]

X = filtered_df[['CO(GT)']]
y = filtered_df['C6H6(GT)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_1 = LinearRegression()
model_1.fit(X_train, y_train)
y_pred_1 = model_1.predict(X_test)

r2_1 = r2_score(y_test, y_pred_1)
mse_1 = mean_squared_error(y_test, y_pred_1)

print("1. Базовая линейная регрессия:")
print(f"R-квадрат: {r2_1:.2f}")
print(f"MSE: {mse_1:.2f}")

1. Базовая линейная регрессия:
R-квадрат: 0.87
MSE: 6.58


# 2. Множественная линейная регрессия

In [15]:
fdf = df
fdf = fdf[fdf['C6H6(GT)'].notnull()]
fdf = fdf[fdf['T'].notnull()]
fdf = fdf[fdf['RH'].notnull()]
fdf = fdf[fdf['NO2(GT)'].notnull()]
fdf = fdf[fdf['CO(GT)'].notnull()]
X = fdf[['C6H6(GT)', 'T', 'RH', 'NO2(GT)']]  # Пример набора признаков
y = fdf['CO(GT)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_2 = LinearRegression()
model_2.fit(X_train, y_train)
y_pred_2 = model_2.predict(X_test)

r2_2 = r2_score(y_test, y_pred_2)
mse_2 = mean_squared_error(y_test, y_pred_2)

print("\n2. Множественная линейная регрессия:")
print(f"R-квадрат: {r2_2:.2f}")
print(f"MSE: {mse_2:.2f}")


2. Множественная линейная регрессия:
R-квадрат: 0.90
MSE: 0.18


# 3. Стандартизация признаков и линейная регрессия

In [16]:
fdf = df
fdf = fdf[fdf['C6H6(GT)'].notnull()]
fdf = fdf[fdf['T'].notnull()]
fdf = fdf[fdf['RH'].notnull()]
fdf = fdf[fdf['NO2(GT)'].notnull()]
fdf = fdf[fdf['CO(GT)'].notnull()]
X = fdf[['C6H6(GT)', 'T', 'RH', 'NO2(GT)']]  # Пример набора признаков
y = fdf['CO(GT)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_3 = LinearRegression()
model_3.fit(X_train_scaled, y_train)
y_pred_3 = model_3.predict(X_test_scaled)

r2_3 = r2_score(y_test, y_pred_3)
mse_3 = mean_squared_error(y_test, y_pred_3)

print("\n3. Стандартизация и линейная регрессия:")
print(f"R-квадрат: {r2_3:.2f}")
print(f"MSE: {mse_3:.2f}")


3. Стандартизация и линейная регрессия:
R-квадрат: 0.90
MSE: 0.18


# 4. Линейная регрессия с L1 регуляризацией (Lasso)

In [ ]:
fdf = df.dropna()
X = fdf.drop(['CO(GT)', 'Date', 'Time'], axis=1)  # Все признаки, кроме целевой
y = fdf['CO(GT)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lasso_cv = LassoCV(cv=5)  # Кросс-валидация для выбора alpha
lasso_cv.fit(X_train_scaled, y_train)

model_4 = Lasso(alpha=lasso_cv.alpha_)
model_4.fit(X_train_scaled, y_train)
y_pred_4 = model_4.predict(X_test_scaled)

r2_4 = r2_score(y_test, y_pred_4)
mse_4 = mean_squared_error(y_test, y_pred_4)
non_zero_coeffs = np.sum(model_4.coef_ != 0)

print("\n4. Lasso регрессия:")
print(f"R-квадрат: {r2_4:.2f}")
print(f"MSE: {mse_4:.2f}")
print(f"Оптимальный alpha: {lasso_cv.alpha_:.4f}")
print(f"Ненулевые коэффициенты: {non_zero_coeffs}")

TypeError: got an unexpected keyword argument 'n_samples'

Незначительное улучшение по сравнению с моделью 2 (R-квадрат = 0.69, MSE = 0.18). Lasso регуляризация, вероятно, отбросила некоторые менее важные признаки, что привело к небольшому улучшению обобщающей способности.

# 5. Линейная регрессия с L2 регуляризацией (Ridge)

In [34]:
X = df.drop(['alcohol', 'target'], axis=1)
y = df['alcohol']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ridge_cv = RidgeCV(cv=5)
ridge_cv.fit(X_train_scaled, y_train)

model_5 = Ridge(alpha=ridge_cv.alpha_)
model_5.fit(X_train_scaled, y_train)
y_pred_5 = model_5.predict(X_test_scaled)

r2_5 = r2_score(y_test, y_pred_5)
mse_5 = mean_squared_error(y_test, y_pred_5)

print("\n5. Ridge регрессия:")
print(f"R-квадрат: {r2_5:.2f}")
print(f"MSE: {mse_5:.2f}")
print(f"Оптимальный alpha: {ridge_cv.alpha_:.4f}")


5. Ridge регрессия:
R-квадрат: 0.70
MSE: 0.18
Оптимальный alpha: 10.0000


Результаты очень похожи на Lasso, с небольшим улучшением (R-квадрат = 0.70, MSE = 0.18). Ridge регуляризация также помогла улучшить обобщающую способность, но не так сильно, как можно было ожидать.

# 6. Кросс-валидация для оценки модели (множественная регрессия)

In [35]:
X = df[['flavanoids', 'proline', 'magnesium', 'color_intensity']]
y = df['alcohol']

model_6 = LinearRegression()
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # K-fold кросс-валидация

r2_scores = cross_val_score(model_6, X, y, cv=kf, scoring='r2')
mse_scores = cross_val_score(model_6, X, y, cv=kf, scoring='neg_mean_squared_error') # negative MSE

print("\n6. Кросс-валидация:")
print(f"Средний R-квадрат: {r2_scores.mean():.2f}")
print(f"Средний MSE: {-mse_scores.mean():.2f}") # Возвращаем положительное значение MSE



6. Кросс-валидация:
Средний R-квадрат: 0.51
Средний MSE: 0.31


# 8. Полиномиальная регрессия

In [36]:
X = df[['flavanoids', 'proline']] # Используем 2 признака для примера
y = df['alcohol']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

poly = PolynomialFeatures(degree=2) # Создаем полиномиальные признаки степени 2
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

model_8 = LinearRegression()
model_8.fit(X_train_poly, y_train)
y_pred_8 = model_8.predict(X_test_poly)

r2_8 = r2_score(y_test, y_pred_8)
mse_8 = mean_squared_error(y_test, y_pred_8)

print("\n8. Полиномиальная регрессия:")
print(f"R-квадрат: {r2_8:.2f}")
print(f"MSE: {mse_8:.2f}")



8. Полиномиальная регрессия:
R-квадрат: 0.52
MSE: 0.29


Хуже, чем все модели, кроме базовой (R-квадрат = 0.52, MSE = 0.29). Это может указывать на то, что зависимость между признаками и содержанием алкоголя не является сильно нелинейной, или что выбранная степень полинома (2) не является оптимальной, или же произошёл перебор признаков, что привело к ухудшению результатов.


# 9. Сравнение моделей и интерпретация результатов

In [37]:
print("\n9. Сравнение моделей:")
print("Модель 1 (базовая): R-квадрат =", r2_1, ", MSE =", mse_1)
print("Модель 2 (множественная): R-квадрат =", r2_2, ", MSE =", mse_2)
print("Модель 3 (стандартизация): R-квадрат =", r2_3, ", MSE =", mse_3)
print("Модель 4 (Lasso): R-квадрат =", r2_4, ", MSE =", mse_4)
print("Модель 5 (Ridge): R-квадрат =", r2_5, ", MSE =", mse_5)
print("Модель 8 (полиномиальная): R-квадрат =", r2_8, ", MSE =", mse_8)

# Интерпретация (пример):
print("\nИнтерпретация (пример):")
print("Коэффициенты модели 2 (множественная регрессия):", model_2.coef_) # Вывод коэффициентов
print("Это позволяет оценить вклад каждого признака в предсказание уровня алкоголя.")



9. Сравнение моделей:
Модель 1 (базовая): R-квадрат = 0.07847669754417019 , MSE = 0.5501824043757246
Модель 2 (множественная): R-квадрат = 0.6599284464234272 , MSE = 0.20303489288651488
Модель 3 (стандартизация): R-квадрат = 0.6599284464234265 , MSE = 0.20303489288651527
Модель 4 (Lasso): R-квадрат = 0.6927030612445944 , MSE = 0.18346727442614258
Модель 5 (Ridge): R-квадрат = 0.6956463363941323 , MSE = 0.18171003378535067
Модель 8 (полиномиальная): R-квадрат = 0.5176336077427277 , MSE = 0.28799000608546255

Интерпретация (пример):
Коэффициенты модели 2 (множественная регрессия): [ 0.06722727  0.00131362 -0.00255745  0.14317662]
Это позволяет оценить вклад каждого признака в предсказание уровня алкоголя.


**Выводы:**

•  Использование нескольких признаков значительно улучшает предсказание содержания алкоголя по сравнению с использованием только одного признака.
•  Регуляризация (Lasso и Ridge) приводит к небольшому улучшению обобщающей способности модели, за счет отбрасывания или уменьшения влияния менее важных признаков.
•  В данном случае, нет сильной нелинейной зависимости между признаками и целевой переменной, так как полиномиальная регрессия показала результаты хуже, чем линейные модели с регуляризацией.
•  Модели с регуляризацией (Lasso и Ridge) показали лучшие результаты, чем обычная множественная линейная регрессия, что может указывать на наличие мультиколлинеарности в данных.